In [2]:
import glob
import os
import re
import numpy as np
import pprint

In [13]:
subgoals = glob.glob("./results/*.subgoal.txt")
w = open("phrase_axioms.txt", "w")
for subgoal in subgoals:
    fileid = re.search("./results/sick_trial_(.*).subgoal.txt", subgoal).group(1)
    f = open(subgoal, "r")
    infos=f.readlines()
    f.close()
    subgoalflg = 0
    premises, content_subgoals, relation_subgoals, premise_preds, subgoal_preds, premise_args, subgoal_args= [], [], [], [], [], [], []
    for info in infos:
        if re.search("defined", info):
            premises, content_subgoals, relation_subgoals, premise_preds, subgoal_preds, premise_args, subgoal_args= [], [], [], [], [], [], []
            break
        if re.search("H[0-9]? : ", info) and not "True" in info:
            premise = re.search("H[0-9]? :(.*)\n", info).group(1)
            if premise != "":
                premises.append(premise)
                premise_list = premise.split()
                if len(premise_list) > 0:
                    premise_pred = premise_list[0]
                    premise_preds.append(premise_pred)
                    premise_arg = " ".join(premise_list[1:])
                    premise_args.append(premise_arg)
        if re.search("============================", info) or re.search("subgoal [0-9]* is:", info):
            subgoalflg = 1
            continue
        if subgoalflg == 1:
            if "=" in info:
                #keep realtional subgoals
                relation_subgoals.append(info.strip())
            elif "False" in info:
                #assume False as relational subgoals
                relation_subgoals.append(info.strip())
            else:
                subgoal = info.strip()
                if subgoal != "" and subgoal not in content_subgoals:
                    # check if subgoal is not None or duplicated
                    content_subgoals.append(subgoal)
                    subgoal_list = subgoal.split()
                    if len(subgoal_list) > 0:
                        subgoal_pred = subgoal_list[0]
                        subgoal_preds.append(subgoal_pred)
                        subgoal_arg = " ".join(subgoal_list[1:])
                        subgoal_args.append(subgoal_arg)
    np_premises = np.array(premises)
    np_subgoals = np.array(content_subgoals)           
    for subgoal_arg in subgoal_args:
        phrase_flg = 0
        if subgoal_arg in premise_args:
            #perfect match
            match_premise_index = [i for i, x in enumerate(premise_args) if x == subgoal_arg]
            match_subgoal_index = [i for i, x in enumerate(subgoal_args) if x == subgoal_arg]
            w.write("id: {0}\tperfect_pair_candidate\tpremise: {1}\tsubgoal: {2}\n".format(fileid, np_premises[match_premise_index], np_subgoals[match_subgoal_index]))
            phrase_flg = 1
            continue
        else:
            #partial match
            for premise_arg in premise_args:
                if premise_arg in subgoal_arg:
                    match_premise_index = [i for i, x in enumerate(premise_args) if x == premise_arg]
                    match_subgoal_index = [i for i, x in enumerate(subgoal_args) if x == subgoal_arg]
                    w.write("id:{0}\tpartial_pair_candidate\tpremise: {1}\tsubgoal: {2}\n".format(fileid, np_premises[match_premise_index], np_subgoals[match_subgoal_index]))
                    phrase_flg = 1
                    break
        if phrase_flg == 0:
            match_subgoal_index = [i for i, x in enumerate(subgoal_args) if x == subgoal_arg]
            #make subgoal sets who share the same existential variables
            w.write("id:{0}\tno_candidate\tpremise: {1}\tsubgoal: {2}\n".format(fileid, premises, np_subgoals[match_subgoal_index]))

w.close()
    #pprint.pprint(premises)        
    #print("premises:{0}, subgoals:{1}, premise_pred:{2}, subgoal_pred:{3}, premise_arg:{4}, subgoal_arg:{5}"\
    #    .format(premises, content_subgoals, premise_preds, subgoal_preds, premise_args, subgoal_args))